In [1]:
import mission_to_mars
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd
import json
import mars_test
from flask_pymongo import PyMongo
from flask import Flask, render_template, redirect

In [32]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/mission_to_mars_app"
mongo = PyMongo(app)

In [36]:
news = mongo.db.news.find_one()
facts = mongo.db.facts.find_one()

In [2]:
f=mission_to_mars.scrape_facts()

0
1
2
3
4
5
6
7
8


In [8]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path)
url_facts='https://space-facts.com/mars/'
browser.visit(url_facts)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article=soup.table.find_all('tr')

table_data=[]
for i in range(len(article)):
    p=article[i].find('td', class_='column-1').text
    v=article[i].find('td', class_='column-2').text
    table_data.append({'Property':p,'Value':v})
browser.quit()

In [5]:
for thing in f:
    print(thing['Property'] + thing['Value'])

Equatorial Diameter:6,792 km
Polar Diameter:6,752 km
Mass:6.39 × 10^23 kg (0.11 Earths)
Moons:2 (Phobos & Deimos)
Orbit Distance:227,943,824 km (1.38 AU)
Orbit Period:687 days (1.9 years)
Surface Temperature: -87 to -5 °C
First Record:2nd millennium BC
Recorded By:Egyptian astronomers


In [8]:
f_dict={'Data':f}
f_dict

{'Data': [{'Property': 'Equatorial Diameter:', 'Value': '6,792 km'},
  {'Property': 'Polar Diameter:', 'Value': '6,752 km'},
  {'Property': 'Mass:', 'Value': '6.39 × 10^23 kg (0.11 Earths)'},
  {'Property': 'Moons:', 'Value': '2 (Phobos & Deimos)'},
  {'Property': 'Orbit Distance:', 'Value': '227,943,824 km (1.38 AU)'},
  {'Property': 'Orbit Period:', 'Value': '687 days (1.9 years)'},
  {'Property': 'Surface Temperature: ', 'Value': '-87 to -5 °C'},
  {'Property': 'First Record:', 'Value': '2nd millennium BC'},
  {'Property': 'Recorded By:', 'Value': 'Egyptian astronomers'}]}

In [11]:
for fact in f_dict['Data']:
    print(fact)

{'Property': 'Equatorial Diameter:', 'Value': '6,792 km'}
{'Property': 'Polar Diameter:', 'Value': '6,752 km'}
{'Property': 'Mass:', 'Value': '6.39 × 10^23 kg (0.11 Earths)'}
{'Property': 'Moons:', 'Value': '2 (Phobos & Deimos)'}
{'Property': 'Orbit Distance:', 'Value': '227,943,824 km (1.38 AU)'}
{'Property': 'Orbit Period:', 'Value': '687 days (1.9 years)'}
{'Property': 'Surface Temperature: ', 'Value': '-87 to -5 °C'}
{'Property': 'First Record:', 'Value': '2nd millennium BC'}
{'Property': 'Recorded By:', 'Value': 'Egyptian astronomers'}


In [49]:
facts['Fact']

['Equatorial Diameter:',
 'Polar Diameter:',
 'Mass:',
 'Moons:',
 'Orbit Distance:',
 'Orbit Period:',
 'Surface Temperature: ',
 'First Record:',
 'Recorded By:']

In [51]:
for i in range(9):
    print(facts['Fact'][i])
    print(facts['Value'][i])

Equatorial Diameter:
6,792 km
Polar Diameter:
6,752 km
Mass:
6.39 × 10^23 kg (0.11 Earths)
Moons:
2 (Phobos & Deimos)
Orbit Distance:
227,943,824 km (1.38 AU)
Orbit Period:
687 days (1.9 years)
Surface Temperature: 
-87 to -5 °C
First Record:
2nd millennium BC
Recorded By:
Egyptian astronomers


In [13]:
data2=pd.DataFrame({"Title": [data.Title], "Body": [data.Body]})
data2

,Title,Body
0,NASA's Perseverance Rover Is Midway to Mars,Sometimes half measures can be a good thing – ...


In [12]:
data.Title

"NASA's Perseverance Rover Is Midway to Mars "

In [11]:
data=mission_to_mars.mars_news()
#data=mars_test.mars_news2()
#data=mars_news()
data.head()

Title         NASA's Perseverance Rover Is Midway to Mars 
Body     Sometimes half measures can be a good thing – ...
Name: 0, dtype: object

In [16]:
url_mars_news='https://mars.nasa.gov/news/'
featured_url="<a href=\""+url_mars_news+"\"></a>"
featured_url

'<a href="https://mars.nasa.gov/news/"></a>'

In [17]:
mission_to_mars.scrape_images()

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24182-640x350.jpg'

In [18]:
hemi= mission_to_mars.scrape_hemi()

In [27]:
hemi['Img_url'].tolist()

['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [2]:
def mars_news():
    #Parse Mars News
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path)
    url_mars_news='https://mars.nasa.gov/news/'
    browser.visit(url_mars_news)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    article=soup.find_all('ul', class_='item_list')[0].find_all('li',class_='slide')

    title=[]
    news=[]
    for i in range(len(article)):
        title.append(article[i].find('div',class_='content_title').text)
        news.append(article[i].find('div',class_='article_teaser_body').text)
    
    news=pd.DataFrame({"Title": title, "Body": news})

    browser.quit()

    return news